# Name: Salvador Cornejo
# Assignment: 3
# Course: CS4662
# Date Due: 04/05/2023

### Problem 2

#### Getting the features from our file. Which are images in this case. We are reshaped into a 1D array using .reshape(-1) since thats what we want, a 1D array of the pixels

In [200]:
import matplotlib.image as mpimg

features = []
for x in range(400):
    img = mpimg.imread("Face/" + str(x) + ".jpg")
    imgArr = np.asarray(img).reshape(-1)
    features.append(imgArr)
X = np.array(features)

#### Matrix of our features(images) 

In [201]:
X

array([[186, 183, 179, ...,   9,   3,   5],
       [204, 198, 194, ..., 120, 164, 167],
       [ 86,  79,  82, ..., 189, 185, 141],
       ...,
       [163, 177, 193, ...,  44,  48,  53],
       [145, 147, 151, ..., 132, 114, 117],
       [ 91,  74,  66, ...,  75,  44,  46]], dtype=uint8)

#### Getting the labels

In [202]:
path = "Face/label.csv"
y = pd.read_csv(path)

#### Printing our labels

In [203]:
y

,Label
0,13
1,30
2,34
3,19
4,24
...,...
395,32
396,19
397,11
398,4


#### Normalizing each column of our features.

In [204]:
from sklearn import preprocessing
X_normalized = preprocessing.scale(X) # no for loop?

#### Spliting our data into training and testing sets

In [205]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.25, random_state=5)

#### The dimensionality of the data samples is 4096. We use PCA (Principal Component Analysis) to reduce the dimensionality from 4096 to 50 (i.e. only k=50 principal components). We train our PCA model on our training set only, and then we use this fitted model to “transform” both training and testing sets. Number of columns in our testing and training sets should be 50

In [206]:
from  sklearn.decomposition  import  PCA

k = 50  #  k  is the number of components (new features) after dimensionality reduction

my_pca = PCA(n_components = k).fit(X_train)

# X_train is feature matrix of training set before dimensionality reduction, 

# X_train_New is feature matrix of training set after dimensionality reduction:

X_train_new = my_pca.fit_transform(X_train)

X_test_new = my_pca.transform(X_test)

#### Checking what type is our X_train_new

In [207]:
print(type(X_train_new))

<class 'numpy.ndarray'>


#### Checking the number of columns in our testing and training sets. They should be 50.

In [208]:
print(X_train_new.shape)
X_test_new.shape

(300, 50)


(100, 50)

#### We design and train non-linear SVM classifier with "RBF Kernal" to recognize the face based on the training dataset that we build.

In [209]:
from sklearn import svm

my_svm_svc = svm.SVC(C=1, kernel='rbf', gamma=0.0005, random_state=1)

# from sklearn.svm import SVC

# my_svm_svc = SVC(C=1, kernel='rbf', gamma=0.0005, random_state=1)

#### Training our svm SRC model with our `X_train_new` and our `y_train`. As mentioned `X_train_new` is our features that we reduced the dimensionality from the originsl training feature set

In [210]:
my_svm_svc.fit(X_train_new, y_train.values.ravel())

SVC(C=1, gamma=0.0005, random_state=1)

#### We make prediction on our SVC model with our `X_test_new`. Our `X_test_new` is the testing samples with the projection to our eigen vectors

In [211]:
y_predict = my_svm_svc.predict(X_test_new)
print('Prediction: ', y_predict)
print('Size of our prediction: ', len(y_predict))

Prediction:  [16 32 39 22 29 38 21 12 23 28  9 13 29 21 10  1 36 11 39 25  8 26 19 12
 35  7 21 20  8 14  7 33 23  0 20  1 38 25  8 23 39  2 10 13 38 25 26 18
  4 36 26 17 13 16 38 36  4 36  5  6 14  9 13  7  9 22  6  6 24  0  4 31
 31 37 14 22 16 34 11 15 15  0  2 25  5 34 11 18 22 11 12  1 16 24 36 33
 32 13 38 20]
Size of our prediction:  100


#### We now calculate the the accuracy of our prediction with our actual labels

In [212]:
from sklearn.metrics import accuracy_score
predictonAccuracy = accuracy_score(y_test, y_predict)
print('Accuracy: ', predictonAccuracy)

Accuracy:  0.91


#### We calculate and report the Confusion Matrix

In [213]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predict)
print('Confusion Matrix: ', cm)
print('Shape of Confusion Matrix', cm.shape)

Confusion Matrix:  [[3 0 0 ... 0 0 0]
 [0 3 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 4 0]
 [0 0 0 ... 0 0 1]]
Shape of Confusion Matrix (37, 37)


#### We use GridSearchCV to find the best value for parameter C in our SVM with scoring='accuracy'. We search in this list: [0.1, 1, 10, 100, 1e3, 5e3, 1e4, 5e4, 1e5]. We want to use cross-validation method (GridSearchCV) to find the best C. We have to perform another PCA: for the sake of simplicity, this time we can do the PCA on the entire dataset after normalization (X_normalized_pca = my_pca.fit_transform(X_normalized)), where X_normalize is the entire dataset after normalization, and then use it in GridSearchCV with 10-fold cross validation to find C.

In [214]:
from sklearn.model_selection import GridSearchCV
new_SVC = svm.SVC(C=1, kernel='rbf', gamma=0.0005, random_state=1)



k = 50  #  k  is the number of components (new features) after dimensionality reduction

my_pca = PCA(n_components = k)



param_grid = {'C': [0.1, 1, 10, 100, 1e3, 5e3, 1e4, 5e4, 1e5], }
X_normalize = preprocessing.scale(X)
X_normalized_pca = my_pca.fit_transform(X_normalize)

grid = GridSearchCV(new_SVC, param_grid, cv=10, scoring= 'accuracy')
grid.fit(X_normalized_pca, y)

/Users/salvadorcornejo1/Desktop/anaconda/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/salvadorcornejo1/Desktop/anaconda/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/salvadorcornejo1/Desktop/anaconda/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/salvadorcornejo1/Desktop/anaconda/anaconda3/lib/python3.8/site-packages/sklearn/utils/vali

/Users/salvadorcornejo1/Desktop/anaconda/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/salvadorcornejo1/Desktop/anaconda/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/salvadorcornejo1/Desktop/anaconda/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/salvadorcornejo1/Desktop/anaconda/anaconda3/lib/python3.8/site-packages/sklearn/utils/vali

/Users/salvadorcornejo1/Desktop/anaconda/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/salvadorcornejo1/Desktop/anaconda/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/salvadorcornejo1/Desktop/anaconda/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/salvadorcornejo1/Desktop/anaconda/anaconda3/lib/python3.8/site-packages/sklearn/utils/vali

/Users/salvadorcornejo1/Desktop/anaconda/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/salvadorcornejo1/Desktop/anaconda/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=10, estimator=SVC(C=1, gamma=0.0005, random_state=1),
             param_grid={'C': [0.1, 1, 10, 100, 1000.0, 5000.0, 10000.0,
                               50000.0, 100000.0]},
             scoring='accuracy')

#### Results of best C value and Accuracy.

In [215]:
print('Best C value: ', grid.best_params_)
print('Accuracy: ', grid.best_score_)

Best C value:  {'C': 10}
Accuracy:  0.9649999999999999
